In [3]:
class Bio_comp:
    '''
    class Bio_Maredat_comp(runname,resultpath,savepath,meshpath,ncfileMaredat,FT,first_year,last_year,
                 mapproj='pc',savefig=False, verbose=False, output=False, 
                            plotting=True, Taylor=True)
    '''
    
    def __init__(self,resultpath,savepath,mesh,meshpath,ncfileMaredat,FT,first_year,last_year,
                 mapproj='pc',runid='fesom',
                 savefig=False,output=False,plotting=True,verbose=False,Taylor=True):

        self.runid = runid
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.meshpath = meshpath
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.savefig = savefig
        self.ncfileMaredat = ncfileMaredat
        self.FT = FT
        self.verbose = verbose
        self.output = output
        self.plotting = plotting
        self.Taylor = Taylor

        import matplotlib.pyplot as plt
        import matplotlib.colors as colors
        import numpy as np
        #from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle
        import scipy.io as spio
        import cartopy.feature as cfeature
        from netCDF4 import Dataset
        import math 
        import xarray as xr

        import pyfesom2 as pf
        
        from Py_f2recom_toolbox import plt_Taylor_norm
        
        import Py_f2recom_toolbox as stx
        
        import warnings
        warnings.filterwarnings('ignore')
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
            
        self.mapproj = pf.get_proj(self.mapproj)

        if(self.verbose):
            print('Processing {0}'.format(self.resultpath))

            
# load Maredat -------------------------------------------------------------------------------
    # for Phaeocystis Maredat: Extract Arctic and Antarctic Data
        if ncfileMaredat == '/albedo/home/simuel001/master_thesis/py_f2recom_4phy/eval_maredat_phaeo/PANGAEA/MarEDat20120424Phaeocystis.nc':
            

            ncfileMaredat = xr.open_dataset(self.ncfileMaredat, decode_times=False)
            
            mask = ((ncfileMaredat.LATITUDE < 45) & (ncfileMaredat.LATITUDE > -45))

            filtered_data = ncfileMaredat.where(~mask, drop=False)

            # saving maredat data with extracted latitude data:
            
            path = '/albedo/home/simuel001/master_thesis/py_f2recom_4phy/eval_maredat_phaeo/PANGAEA/MarEDat20120424Phaeocystis_filtered.nc'
            filtered_data.to_netcdf(path=path)

            # continue with extracted data: 
            lat_maredat, lon_maredat, maredat_layered_sum, maredat_layered_mean = stx.maredat_load(path)

        else:
            lat_maredat, lon_maredat, maredat_layered_sum, maredat_layered_mean = stx.maredat_load(self.ncfileMaredat)

            
            
# load FESOM mesh -------------------------------------------------------------------------------------
        years = np.arange(self.fyear, self.lyear+1,1)
        
        lon_fesom = mesh.x2
        lat_fesom = mesh.y2       
            
            
# load FESOM mesh diag -------------------------------------------------------------------------------
        meshdiag=resultpath+'/'+runid+'.mesh.diag.nc'
        #!ncdump -h $meshdiag
        diag = pf.get_meshdiag(mesh,meshdiag=meshdiag,runid=runid)
        mesh_depths = diag['nz'].values

        if self.verbose:
            stx.check_plot_mesh(mesh, plot_globe = False)
                        
            
# load FESOM Diatom C data -------------------------------------------------------------------------------------
        fesom_layered_sum, fesom_layered_mean = stx.fesom_to_maredat_levels(self.resultpath, self.runid, years, mesh, lon_maredat, lat_maredat, self.FT)
    
            
# mask FESOM data for gridpoints without Maredat data ----------------------------------------------------------
        fesom_layered_sum_masked  = stx.mask_model_with_maredat(fesom_layered_sum,maredat_layered_sum)
        fesom_layered_mean_masked = stx.mask_model_with_maredat(fesom_layered_mean,maredat_layered_mean)
                   
            
# Taylor statistics ---------------------------------------------------------------------------------       
        fig_1, sdev_1, crmsd_1, ccoef_1, fig_2, sdev_2, crmsd_2, ccoef_3, fig_3, sdev_3, crmsd_3, ccoef_3, fig_4, sdev_4, crmsd_4, ccoef_4 = stx.plot_Taylor(maredat_layered_sum,fesom_layered_sum_masked,self.FT,self.savefig)
        

# Barplots ----------------------------------------------------------------------------------------  
        stx.barplots(fesom_layered_sum_masked, maredat_layered_sum, lon_maredat, lat_maredat, self.FT, self.savefig)
    
    
# plot Maredat and (unfiltered) FESOM next to each other ---------------------------------------------------
#        figlabel = '' # label used for plotting
#        savelabel = '' # extra label used for saving output
#        mapproj='pc' # other projection than for the remaining Master script

#        stx.plot_2cols(fesom_layered_sum,figlabel,savelabel, maredat_layered_sum, lon_maredat, lat_maredat, mapproj, years, self.savefig, self.savepath)


# Scatter plot ----------------------------------------------------------------------------------------        
#        stx.plot_scatter(fesom_layered_sum_masked, figlabel, savelabel, maredat_layered_sum, years, self.savefig, self.savepath, ccoef_1)



    